In [1]:
! pip install transformers
! pip install datasets
!pip install torchmetrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.2 MB/s 
     |████████████████████████████████| 6.6 MB 66.8 MB/s 
     |████████████████████████████████| 120 kB 94.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 8.0 MB/s 
     |████████████████████████████████| 212 kB 57.7 MB/s 
     |████████████████████████████████| 115 kB 62.8 MB/s 
     |████████████████████████████████| 127 kB 58.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 5.7 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time

from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.utils import shuffle
import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation

Mounted at /content/drive
/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation


In [5]:
pd.set_option('display.max_rows', None)

In [6]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
df2 = pd.read_json('ERAI_Dataset_pairwise_test_with_translation.json')
df.reset_index(drop = True,inplace = True)
df = df.dropna(axis = 0)



df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))


df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
df['post2']=df['post2'].apply(lambda x:x.replace('\n',''))


In [7]:
PRETRAINED_MODEL_NAME =  "bert-base-chinese"

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [8]:

def generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2):
  
  
  first_all_value = pd.concat([mpp_value1,ml_value1],axis=1)
  second_all_value = pd.concat([mpp_value2,ml_value2],axis=1)
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'mpp_label':torch.tensor(mpp_label.values,dtype = torch.long),
      'ml_label':torch.tensor(ml_label.values,dtype = torch.long),
      'first_all_value': torch.tensor(first_all_value.values, dtype=torch.float),
      'second_all_value': torch.tensor(second_all_value.values, dtype=torch.float),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

    

In [9]:
class BertNerClassifier(nn.Module):

  def __init__(self, n_classes_cls,n_classes_reg):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained("bert-base-chinese")
    self.drop = nn.Dropout(p=0.1)
    self.out_cls = nn.Linear(self.bert.config.hidden_size, n_classes_cls)

    self.out_reg = nn.Linear(self.bert.config.hidden_size, n_classes_reg)
    self.sigmoid = nn.Sigmoid()

  
  def forward(self, input_ids, attention_mask,token_type_ids,max_len,length1,length2):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids = token_type_ids

    )
    cls_output = []
    sent1_output = []
    sent2_output = []
    
    output = self.drop(output[0])

    for last_output,len1,len2 in zip(output,length1,length2):
      cls_output.append(last_output[0])
      
      sent1_output.append(torch.mean(last_output[1:(len1+1)],dim=0))
      sent2_output.append(torch.mean(last_output[len1+2:(len1+2+len2)],dim=0))

    cls_output = torch.stack(cls_output)
    sent1_output = torch.stack(sent1_output)
    
    sent2_output = torch.stack(sent2_output)
    
    cls_output = self.out_cls(cls_output)
    
    
    
    return self.sigmoid(cls_output),self.out_reg(sent1_output),self.out_reg(sent2_output)

In [10]:
 
class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']
      self.mpp_label = data['mpp_label']
      self.ml_label = data['ml_label']
      self.first_all_value = data['first_all_value']
      self.second_all_value = data['second_all_value']
      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.mpp_label)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.mpp_label[index],self.ml_label[index],self.first_all_value[index],self.second_all_value[index],self.first_sen_len[index],self.second_sen_len[index]

In [11]:
def generate_dataloader(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2,batch_size):
  data = generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader


In [13]:
EPOCHS = 50

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_reg = nn.MSELoss().to(device)
loss_cls = nn.CrossEntropyLoss().to(device)


In [22]:
def train_and_eval(model, 
  train_loader, 
  validation_loader, 
  optimizer,  
  device,
  epoch,
  t_examples,
  v_examples,
  mpp_flag = True 
  ):
  model.train()
  max_len = 256

  acccuracy_state = 0
  for epo in range(epoch):
    
    train_loss = []
    
    validation_loss = []
    
    correct_predictions_train = 0
    correct_predictions_validation = 0
    
   
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      token_type_ids = data[2].to(device)
      
      mpp_targets = data[3].to(device)
      ml_targets = data[4].to(device)
      first_targets = data[5].to(device)
      second_targets = data[6].to(device)
      first_sen_len = data[7].to(device)
      second_sen_len = data[8].to(device)
      
      

      output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
      )
      
      
      
      if mpp_flag == True:
        loss1 = loss_cls(output_cls.view(-1, 2),mpp_targets).to(device)
        
      else:
        loss1 = loss_cls(output_cls.view(-1, 2),ml_targets).to(device)
        

     
      loss = loss1

      train_loss.append(loss.item())
      
      _, preds = torch.max(output_cls, dim=1)
 
      correct_predictions_train += torch.sum(preds == mpp_targets)

      
    
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval()

    with torch.no_grad():
      for data in validation_loader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        mpp_targets = data[3].to(device)
        ml_targets = data[4].to(device)
        first_targets = data[5].to(device)
        second_targets = data[6].to(device)
        first_sen_len = data[7].to(device)
        second_sen_len = data[8].to(device)
      

        output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
        )
        if mpp_flag == True:
          loss1 = loss_cls(output_cls.view(-1, 2),mpp_targets).to(device)
          
        else:
          loss1 = loss_cls(output_cls.view(-1, 2),ml_targets).to(device)
          
        
        
        loss = loss1
        validation_loss.append(loss.item())

        _, preds = torch.max(output_cls, dim=1)
 
        correct_predictions_validation += torch.sum(preds == mpp_targets)

   
    final_train_loss = np.mean(train_loss)
 
    final_validation_loss = np.mean(validation_loss)
    
    validation_accuracy = correct_predictions_validation.double() / v_examples
    train_accuracy = correct_predictions_train.double() / t_examples

    if(validation_accuracy > acccuracy_state):
      acccuracy_state = validation_accuracy
      if mpp_flag == True:
        torch.save(model.state_dict(), 'mpp_singe_value_state.bin')
      if mpp_flag == False:
        torch.save(model.state_dict(), 'ml_singe_value_state.bin')


    print(
'Epoch [{}] Train Totol Loss: {:.4f}, Train MPP Accuracy: {:.4f}, Valid Total Loss: {:.4f}, Valid MPP Accuracy: {:.4f}'
    .format(epo+1,final_train_loss, train_accuracy, final_validation_loss,validation_accuracy))

In [23]:
random_seed = [171,354,550,667,985]
for i in random_seed:
  
  df = shuffle(df,random_state=i)

  
  train = df[0:156]

  test = df[156:200]

  train_post1 = train['post1_en']
  train_post2 = train['post2_en']
  train_mpp_label = train['MPP_label']
  train_ml_label = train['ML_label']
  train_mpp_value1 = train['MPP1'] 
  train_mpp_value2 = train['MPP2']
  train_ml_value1 = train['ML1']
  train_ml_value2 = train['ML2']

  test_post1 = test['post1_en']
  test_post2 = test['post2_en']
  test_mpp_label = test['MPP_label']
  test_ml_label = test['ML_label']
  test_mpp_value1 = test['MPP1'] 
  test_mpp_value2 = test['MPP2']
  test_ml_value1 = test['ML1']
  test_ml_value2 = test['ML2']

  max_len = 256
  batch_size = 1
  train_dataloader = generate_dataloader(train_post1,train_post2,train_mpp_label,train_ml_label,max_len,train_mpp_value1,train_ml_value1,train_mpp_value2,train_ml_value2,batch_size)
  test_dataloader = generate_dataloader(test_post1,test_post2,test_mpp_label,test_ml_label,max_len,test_mpp_value1,test_ml_value1,test_mpp_value2,test_ml_value2,batch_size)

  label_num_cls = 2
  label_num_reg = 1
  model_mpp= BertNerClassifier(label_num_cls,label_num_reg).to(device)
  model_ml= BertNerClassifier(label_num_cls,label_num_reg).to(device)
  optimizer_mpp = AdamW(model_mpp.parameters(), lr=2e-6, correct_bias=False)
  optimizer_ml = AdamW(model_ml.parameters(), lr=2e-6, correct_bias=False)
  
  
  train_and_eval(model_mpp, 
    train_dataloader, 
    test_dataloader, 
    optimizer_mpp,  
    device,
    EPOCHS,
    len(train),
    len(test),
    )
  train_and_eval(model_ml, 
    train_dataloader, 
    test_dataloader, 
    optimizer_ml,  
    device,
    EPOCHS,
    len(train),
    len(test),
    mpp_flag = False
    )




Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.pr

Epoch [1] Train Totol Loss: 0.6995, Train MPP Accuracy: 0.5256, Valid Total Loss: 0.6835, Valid MPP Accuracy: 0.5750
Epoch [2] Train Totol Loss: 0.6780, Train MPP Accuracy: 0.5769, Valid Total Loss: 0.6866, Valid MPP Accuracy: 0.5750
Epoch [3] Train Totol Loss: 0.5861, Train MPP Accuracy: 0.7756, Valid Total Loss: 0.7401, Valid MPP Accuracy: 0.4500
Epoch [4] Train Totol Loss: 0.4006, Train MPP Accuracy: 0.9744, Valid Total Loss: 0.7904, Valid MPP Accuracy: 0.4750
Epoch [5] Train Totol Loss: 0.3388, Train MPP Accuracy: 0.9872, Valid Total Loss: 0.7858, Valid MPP Accuracy: 0.5000
Epoch [6] Train Totol Loss: 0.3218, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.8026, Valid MPP Accuracy: 0.5000
Epoch [7] Train Totol Loss: 0.3181, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.8120, Valid MPP Accuracy: 0.4750
Epoch [8] Train Totol Loss: 0.3168, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.8184, Valid MPP Accuracy: 0.4750
Epoch [9] Train Totol Loss: 0.3160, Train MPP Accuracy: 1.0000, 

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.pr

Epoch [1] Train Totol Loss: 0.7071, Train MPP Accuracy: 0.5513, Valid Total Loss: 0.6847, Valid MPP Accuracy: 0.5750
Epoch [2] Train Totol Loss: 0.6931, Train MPP Accuracy: 0.5513, Valid Total Loss: 0.6874, Valid MPP Accuracy: 0.5750
Epoch [3] Train Totol Loss: 0.6761, Train MPP Accuracy: 0.5897, Valid Total Loss: 0.6915, Valid MPP Accuracy: 0.5750
Epoch [4] Train Totol Loss: 0.6311, Train MPP Accuracy: 0.6603, Valid Total Loss: 0.6898, Valid MPP Accuracy: 0.5750
Epoch [5] Train Totol Loss: 0.6047, Train MPP Accuracy: 0.6859, Valid Total Loss: 0.6980, Valid MPP Accuracy: 0.5750
Epoch [6] Train Totol Loss: 0.6384, Train MPP Accuracy: 0.6859, Valid Total Loss: 0.6979, Valid MPP Accuracy: 0.5500
Epoch [7] Train Totol Loss: 0.4565, Train MPP Accuracy: 0.8974, Valid Total Loss: 0.7158, Valid MPP Accuracy: 0.5500
Epoch [8] Train Totol Loss: 0.3791, Train MPP Accuracy: 0.9551, Valid Total Loss: 0.7434, Valid MPP Accuracy: 0.5250
Epoch [9] Train Totol Loss: 0.3508, Train MPP Accuracy: 0.9744, 

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.pr

Epoch [1] Train Totol Loss: 0.7046, Train MPP Accuracy: 0.4808, Valid Total Loss: 0.6713, Valid MPP Accuracy: 0.6250
Epoch [2] Train Totol Loss: 0.6893, Train MPP Accuracy: 0.5577, Valid Total Loss: 0.6747, Valid MPP Accuracy: 0.6250
Epoch [3] Train Totol Loss: 0.6550, Train MPP Accuracy: 0.6603, Valid Total Loss: 0.7082, Valid MPP Accuracy: 0.4750
Epoch [4] Train Totol Loss: 0.4784, Train MPP Accuracy: 0.9103, Valid Total Loss: 0.7817, Valid MPP Accuracy: 0.4750
Epoch [5] Train Totol Loss: 0.3501, Train MPP Accuracy: 0.9808, Valid Total Loss: 0.8002, Valid MPP Accuracy: 0.4250
Epoch [6] Train Totol Loss: 0.3218, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.8113, Valid MPP Accuracy: 0.4500
Epoch [7] Train Totol Loss: 0.3182, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.8298, Valid MPP Accuracy: 0.4500
Epoch [8] Train Totol Loss: 0.3167, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.8413, Valid MPP Accuracy: 0.4250
Epoch [9] Train Totol Loss: 0.3159, Train MPP Accuracy: 1.0000, 

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.pr

Epoch [1] Train Totol Loss: 0.6958, Train MPP Accuracy: 0.5705, Valid Total Loss: 0.7598, Valid MPP Accuracy: 0.3750
Epoch [2] Train Totol Loss: 0.6876, Train MPP Accuracy: 0.5449, Valid Total Loss: 0.7434, Valid MPP Accuracy: 0.3500
Epoch [3] Train Totol Loss: 0.6151, Train MPP Accuracy: 0.6859, Valid Total Loss: 0.7600, Valid MPP Accuracy: 0.3750
Epoch [4] Train Totol Loss: 0.4209, Train MPP Accuracy: 0.9551, Valid Total Loss: 0.8214, Valid MPP Accuracy: 0.4250
Epoch [5] Train Totol Loss: 0.3410, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.8813, Valid MPP Accuracy: 0.4000
Epoch [6] Train Totol Loss: 0.3253, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.9089, Valid MPP Accuracy: 0.3750
Epoch [7] Train Totol Loss: 0.3211, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.8922, Valid MPP Accuracy: 0.3250
Epoch [8] Train Totol Loss: 0.3176, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.9267, Valid MPP Accuracy: 0.2750
Epoch [9] Train Totol Loss: 0.3163, Train MPP Accuracy: 1.0000, 

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.pr

Epoch [1] Train Totol Loss: 0.7048, Train MPP Accuracy: 0.4744, Valid Total Loss: 0.7082, Valid MPP Accuracy: 0.3500
Epoch [2] Train Totol Loss: 0.6627, Train MPP Accuracy: 0.6410, Valid Total Loss: 0.7585, Valid MPP Accuracy: 0.3500
Epoch [3] Train Totol Loss: 0.5151, Train MPP Accuracy: 0.8782, Valid Total Loss: 0.8479, Valid MPP Accuracy: 0.3750
Epoch [4] Train Totol Loss: 0.3940, Train MPP Accuracy: 0.9615, Valid Total Loss: 0.7535, Valid MPP Accuracy: 0.5000
Epoch [5] Train Totol Loss: 0.3334, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.7215, Valid MPP Accuracy: 0.5500
Epoch [6] Train Totol Loss: 0.3226, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.7531, Valid MPP Accuracy: 0.5250
Epoch [7] Train Totol Loss: 0.3192, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.7600, Valid MPP Accuracy: 0.5250
Epoch [8] Train Totol Loss: 0.3175, Train MPP Accuracy: 1.0000, Valid Total Loss: 0.7652, Valid MPP Accuracy: 0.5250
Epoch [9] Train Totol Loss: 0.3165, Train MPP Accuracy: 1.0000, 

In [24]:
def generate_test_data(post1,post2,max_len):
  
  
  
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

In [25]:
 
class testDataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']

      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.first_sen_len[index],self.second_sen_len[index]

In [26]:
def generate_test_dataloader(post1,post2,max_len,batch_size):
  data = generate_test_data(post1,post2,max_len)
  data = testDataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader

In [27]:
max_len = 256
batch_size = 1
test_post1 = df2['post1']
test_post2 = df2['post2']
   
test_dataloader = generate_test_dataloader(test_post1,test_post2,max_len,batch_size)

In [31]:
def test_prediction(model, 
  test_dataloader,  
  device,
  ):
  
  test_predictions = []
  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      token_type_ids = data[2].to(device)
      
      
      first_sen_len = data[3].to(device)
      second_sen_len = data[4].to(device)
      
      

      output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
      )
      
      
      _, preds = torch.max(output_cls, dim=1)
      test_predictions.append(preds.item())
     
      
  return test_predictions

In [32]:
model_mpp.load_state_dict(torch.load('mpp_singe_value_state.bin', map_location=torch.device("cuda:0")))
model_ml.load_state_dict(torch.load('ml_singe_value_state.bin', map_location=torch.device("cuda:0")))


<All keys matched successfully>

In [33]:
test_mpp_predictions = test_prediction(model_mpp, 
  test_dataloader, 
  device
  )
test_ml_predictions = test_prediction(model_ml, 
  test_dataloader,
  device
  )

In [34]:
df2['MPP_prediction'] = test_mpp_predictions
df2['ML_prediction'] = test_ml_predictions
df2.to_json('ERAI_pairwise_[aiml]_[1].json')